In [5]:
import tkinter as tk
from tkinter import messagebox
import pymongo
from bson import ObjectId
import json

class CustomButton(tk.Button):
    def __init__(self, master=None, **kwargs):
        super().__init__(master, **kwargs)
        self.configure(highlightthickness=0, bd=0, relief=tk.FLAT)
        self.config(fg="#ecf0f1", activeforeground="#ecf0f1")
        self.config(activebackground="#2c3e50", borderwidth=0, relief=tk.FLAT)

def create_gui():
    def create_entry():
        # Get values from entry fields
        Name = entries[0].get()
    
        # Check if any entry field is empty
        if not all([Name]):
            messagebox.showerror("Error", "Please fill in all fields.")
            return

        # Insert patient data with doctor and address references
        doctor_data = {
            "Name": Name
            
        }

        result = doctor_collection.insert_one(doctor_data)

        messagebox.showinfo("Success", "Entry created successfully.")
    
    
    def execute_query():
        query = search_entry[1].get()  # Get the text from the entry widget
        sort_query = search_entry[2].get()
        if query:
            try:
                query = json.loads(query)
                if sort_query:
                    try:
                        sort_query = eval(sort_query)
                        result = doctor_collection.find(query).sort(sort_query[0],sort_query[1])
                    except:
                        messagebox.showerror("Error", f"Query Sort Error")
                        return
                else:
                    # Find patients based on the constructed query
                    result = doctor_collection.find(query)
                
                count =0
                for doc in result:
                    count +=1
                    result_text.insert(tk.END, f"{doc}\n\n")

                result_label.config(text=f"Search Results / Read Data: ({count})")
            except:
                messagebox.showerror("Error", f"Query Error")
                return
            
            # Execute your query and display results (replace this with your actual query logic)
            messagebox.showinfo("Query Result", f"Query executed: {query}")
        elif sort_query:
            try:
                sort_query = eval(sort_query)
                result = doctor_collection.find().sort(sort_query[0],sort_query[1])
                count =0
                for doc in result:
                    count +=1
                    result_text.insert(tk.END, f"{doc}\n\n")

                result_label.config(text=f"Search Results / Read Data: ({count})")
            except:
                messagebox.showerror("Error", f"Query Sort Error")
                return
        else:
            messagebox.showerror("Error", f"Query not Inserted")
    
    def read_entry():
        # Get values from entry fields
        Name = entries[0].get()

        # Display read data in the results_frame
        result_text.delete(1.0, tk.END)  # Clear previous content

        # Construct query based on filled entries
        query = {}
        if Name:
            query["Name"] = Name
       

        # Find patients based on the constructed query
        result = doctor_collection.find(query)
        count =0
        for doc in result:
            count +=1
            result_text.insert(tk.END, f"{doc}\n\n")
            
        result_label.config(text=f"Search Results / Read Data: ({count})")


    def search_entry():
        ID = search_entry[0].get()
        try:
            result = doctor_collection.find({"_id": ObjectId(ID)})
            # Get the first document (assuming there's only one)
            pat = result[0]

            # Populate the GUI form fields with patient data
            entries[0].insert(0, pat["Name"])
           

            # Display search results
            result_text.delete(1.0, tk.END)  # Clear previous content
            result_text.insert(tk.END, f"Search Results for ID {ID}:\n{pat}")
        except:
            messagebox.showerror("Error", "Doctor not found.")

    def delete_entry():
        ID = search_entry[0].get()
        try:
            result = doctor_collection.find({"_id": ObjectId(ID)})
            doctor_collection.delete_one({"_id": ObjectId(ID)})
            messagebox.showinfo("Delete", "Delete operation performed.")
        except:
            messagebox.showerror("Error", "Not Deleted.")
        

    def update_entry():
        ID = search_entry[0].get()
        try:
            result = doctor_collection.find({"_id": ObjectId(ID)})
            Name = entries[0].get()
            
            doctor_collection.update_one({"_id": ObjectId(ID)}, {"$set": {"Name":Name}})
            messagebox.showinfo("Update", "Update operation performed.")
        except:
            messagebox.showerror("Error", "Not Updated.")
   
    def create_Indexes():
        query = search_entry[3].get()  # Get the text from the entry widget
        if query:
            try:
                doctor_collection.create_index(eval(query))
                messagebox.showinfo("Create Index", "Create Index operation performed.")
            except:
                messagebox.showerror("Error", "Not Indexed.")
        else:
            messagebox.showerror("Error", "Not Query Index is Inserted.")
    def Drop_Indexes():
        name = search_entry[3].get()  # Get the text from the entry widget
        if name:
            try:
                doctor_collection.drop_index(name)
                messagebox.showinfo("Drop_Indexes", f"Drop_Index {name} operation performed.")
            except:
                messagebox.showerror("Error", "Not Indexed.")
        else:
            messagebox.showerror("Error", "Not Index is Dropped.")
    def Show_Indexes():
        result_text.delete(1.0, tk.END)
        result = doctor_collection.list_indexes()
        count = 0
        for doc in result:
            count += 1
            result_text.insert(tk.END, f"{doc}\n\n")

            result_label.config(text=f"Search Results / Read Data: ({count})")
            
    def clear_textboxes():
        result_text.delete(1.0,tk.END)
        for entry in entries:
            entry.delete(0, tk.END)
        for en in search_entry:
            en.delete(0,tk.END)

    root = tk.Tk()
    root.title("CRUD Doctor")
    root.geometry("900x700")  # Increase window height
    root.configure(bg="#34495e")

    form_frame = tk.Frame(root, borderwidth=2, relief=tk.GROOVE, bg="#34495e")
    form_frame.pack(pady=20, padx=20)  # Increase padding

    fields = ["Name:"]
    entries = []

    for i, field_name in enumerate(fields):
        label = tk.Label(form_frame, text=field_name, font=("Arial", 12, "bold"), bg="#34495e", fg="#ecf0f1")
        label.grid(row=i, column=0, padx=10, pady=5, sticky=tk.W)
        entry = tk.Entry(form_frame, font=("Arial", 12), bg="#2c3e50", fg="#ecf0f1", width=50)  # Adjust width here
        entry.grid(row=i, column=1, padx=10, pady=5)
        entries.append(entry)


    button_frame = tk.Frame(root, bg="#34495e")
    button_frame.pack(side=tk.RIGHT, padx=20, pady=20)  # Increase padding

    create_btn = CustomButton(button_frame, text="Create", font=("Arial", 12, "bold"), command=create_entry, bg="#DEB887", relief=tk.RAISED)
    create_btn.pack(pady=5, anchor="e", fill=tk.X, ipadx=10, ipady=5)
    read_btn = CustomButton(button_frame, text="Read", font=("Arial", 12, "bold"), command=read_entry, bg="#DEB887", relief=tk.RAISED)
    read_btn.pack(pady=5, anchor="e", fill=tk.X, ipadx=10, ipady=5)
    search_btn = CustomButton(button_frame, text="Search", font=("Arial", 12, "bold"), command=search_entry, bg="#DEB887", relief=tk.RAISED)
    search_btn.pack(pady=5, anchor="e", fill=tk.X, ipadx=10, ipady=5)
    execute_button = CustomButton(button_frame, text="Execute Query", font=("Arial", 12, "bold"), command=execute_query, bg="#DEB887", relief=tk.RAISED)
    execute_button.pack(pady=5, anchor="e", fill=tk.X, ipadx=10, ipady=5)
    update_btn = CustomButton(button_frame, text="Update", font=("Arial", 12, "bold"), command=update_entry, bg="#DEB887", relief=tk.RAISED)
    update_btn.pack(pady=5, anchor="e", fill=tk.X, ipadx=10, ipady=5)
    delete_btn = CustomButton(button_frame, text="Delete", font=("Arial", 12, "bold"), command=delete_entry, bg="#DEB887", relief=tk.RAISED)
    delete_btn.pack(pady=5, anchor="e", fill=tk.X, ipadx=10, ipady=5)
    clear_button = CustomButton(button_frame, text="Clear", font=("Arial", 12, "bold"), command=clear_textboxes, bg="#DEB887", relief=tk.RAISED)
    clear_button.pack(pady=5, anchor="e", fill=tk.X, ipadx=10, ipady=5)
    Indexes_button = CustomButton(button_frame, text="CreateIndexes", font=("Arial", 12, "bold"), command=create_Indexes, bg="#DEB887", relief=tk.RAISED)
    Indexes_button.pack(pady=5, anchor="e", fill=tk.X, ipadx=10, ipady=5)
    Drop_button = CustomButton(button_frame, text="Drop_Indexes", font=("Arial", 12, "bold"), command=Drop_Indexes, bg="#DEB887", relief=tk.RAISED)
    Drop_button.pack(pady=5, anchor="e", fill=tk.X, ipadx=10, ipady=5)
    Show_button = CustomButton(button_frame, text="Show_Indexes", font=("Arial", 12, "bold"), command=Show_Indexes, bg="#DEB887", relief=tk.RAISED)
    Show_button.pack(pady=5, anchor="e", fill=tk.X, ipadx=10, ipady=5)
   
    
    
    search_frame = tk.Frame(root, borderwidth=2, relief=tk.GROOVE, bg="#34495e")
    search_frame.pack(pady=20, padx=20, fill=tk.X)  # Increase padding



    fields = ["Search by Id:", "Query: ", "Sort Query by:", "CreateIndexes :"]
    search_entry = []

    for i, field_name in enumerate(fields):
        label = tk.Label(search_frame, text=field_name, font=("Arial", 12, "bold"), bg="#34495e", fg="#ecf0f1")
        label.grid(row=i, column=0, padx=10, pady=5, sticky=tk.W)
        entry = tk.Entry(search_frame, font=("Arial", 12), bg="#2c3e50", fg="#ecf0f1", width=60)  # Adjust width here
        entry.grid(row=i, column=1, padx=10, pady=5)
        search_entry.append(entry)

    results_frame = tk.Frame(root, borderwidth=2, relief=tk.GROOVE, bg="#34495e")
    results_frame.pack(pady=20, padx=20, fill=tk.BOTH, expand=True)  # Increase padding

    result_label = tk.Label(results_frame, text="Search Results / Read Data:", font=("Arial", 12, "bold"), bg="#34495e", fg="#ecf0f1")
    result_label.pack(pady=5, padx=10, anchor="w")

    result_text = tk.Text(results_frame, font=("Arial", 12), height=10, bg="#2c3e50", fg="#ecf0f1")
    result_text.pack(pady=5, padx=10, fill=tk.BOTH, expand=True)

    root.mainloop()

# To run in a standalone Python script
if __name__ == "__main__":
    # Replace with your MongoDB connection string if different from localhost
    mongo_uri = "mongodb://localhost:27017/"

    # Connect to MongoDB
    client = pymongo.MongoClient(mongo_uri)

    # Access the database (replace "projectBigData" if needed)
    db = client["projectBigData"]
    patient_collection = db["Patient"]
    health_data_collection = db["Health_Data"]
    diseases_collection = db["disease"]
    diabetes_collection = db["diabetes"]
    arthritis_collection = db["arthritis"]
    doctor_collection = db["Doctors"]
    address_collection = db["Address"]
    create_gui()
